# Оцениваем регрессии на данных HS0 в спецификациях:
- Сырые данные ~ ковид и ограничения + контроль на дамми
- Сырые данные ~ ковид и ограничения + контроль на дамми | инструментальные переменные

In [41]:
library(arrow)
library(stargazer)
library(tidyverse)
library(plotly)
library(fixest)
signif.code = signif.code = c("***" = 0.001, "**" = 0.01, "*" = 0.05, "." = 0.10)

## Сырые данные

Под каждой регрессией вывожу summary с кластеризацией ошибок. В конце блока сравниваю робастность результата по используемым ошибкам.

In [42]:
# Скачиваю данные и переназываю переменные
df = read_parquet('../data/processed_data/trade_hs0_x_covid.parquet.gzip')
# Переменная pair нужна, чтобы считать НАС стандартные ошибки
df = df %>% mutate(pair = paste(i, j))

df$deaths_i = df$deaths
df$deaths_j = df$`deaths:r_j`

df$median_age_i = df$median_age
df$median_age_j = df$`median_age:r_j`

df$average_surface_temperature_i = df$average_surface_temperature
df$average_surface_temperature_j = df$`average_surface_temperature:r_j`

df$C1M_School_closing_i = df$C1M_School_closing
df$C1M_School_closing_j = df$`C1M_School_closing:r_j`

df$population_density_i = df$population_density
df$population_density_j = df$`population_density:r_j`

df$share_1_members_i = df$share_1_members
df$share_1_members_j = df$`share_1_members:r_j`

df$H6M_Facial_Coverings_i = df$H6M_Facial_Coverings
df$H6M_Facial_Coverings_j = df$`H6M_Facial_Coverings:r_j`

df$C8EV_International_travel_controls_i = df$C8EV_International_travel_controls
df$C8EV_International_travel_controls_j = df$`C8EV_International_travel_controls:r_j`
df %>% head()

t,i,j,v,q,v19,q19,country,t:r_i,cases,⋯,C1M_School_closing_i,C1M_School_closing_j,population_density_i,population_density_j,share_1_members_i,share_1_members_j,H6M_Facial_Coverings_i,H6M_Facial_Coverings_j,C8EV_International_travel_controls_i,C8EV_International_travel_controls_j
<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2021,LUX,LBY,968.018,210.173,501.858,127.012,LUX,2021,241.1512329,⋯,1.183562,1.49315071,248.620682,4.055137,33.34,NA,2.0082192,2.827397,2.534247,2.520548
2021,RUS,SEN,1307165.125,3131615.500,789625.305,2232100.632,RUS,2021,138.1677534,⋯,1.290411,0.07671233,8.838488,89.445110,25.69,6.92,2.3150685,3.402740,2.610959,2.561644
2020,SOM,KOR,3136.611,2045.876,5608.268,4002.770,SOM,2020,0.7276796,⋯,1.584700,1.98087430,26.542530,531.109006,1.53,31.75,0.7622951,1.653005,2.101093,2.415301
2020,TUR,CRI,57792.285,98850.453,60858.013,86163.873,TUR,2020,67.6935912,⋯,2.153005,2.40437150,108.343854,98.596161,6.37,12.07,2.2732241,1.218579,2.040984,2.322404
2021,KNA,BIH,0.520,0.008,0.460,0.005,KNA,2021,164.5090411,⋯,NA,1.00000000,179.857692,63.377090,NA,11.50,NA,2.476712,NA,1.000000
2022,AZE,MLT,22060.938,39007.156,12476.490,33493.400,AZE,2022,56.3560822,⋯,2.000000,0.27945206,122.707272,1660.971875,6.77,22.64,0.6575342,2.323288,3.000000,1.263014


In [43]:
# Делаю pulled OLS ради интереса
pulled_ols = feols(log(v/v19) ~ log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j,
 data = df)
pulled_ols %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v/v19)
Observations: 56,739
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                           0.295297   0.023853 12.380068  < 2.2e-16
log(deaths_i)                         0.006162   0.004416  1.395247 1.6472e-01
log(deaths_j)                         0.001219   0.003722  0.327451 7.4372e-01
C8EV_International_travel_controls_i -0.044532   0.011159 -3.990762 9.6932e-05
C8EV_International_travel_controls_j -0.070629   0.010809 -6.534204 6.8081e-10
                                        
(Intercept)                          ***
log(deaths_i)                           
log(deaths_j)                           
C8EV_International_travel_controls_i ***
C8EV_International_travel_controls_j ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.999317   Adj. R2: 0.009329

In [44]:
# FE (т.е. оценка на панельных данных с использованием внутригрупповой дисперсии)
fe_no_iv = feols(log(v/v19) ~ log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j| i + j + year,
 data = df)
fe_no_iv %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v/v19)
Observations: 56,739
Fixed-effects: i: 175,  j: 175,  year: 3
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
log(deaths_i)                        -0.015588   0.006061 -2.571959 0.010948
log(deaths_j)                        -0.003195   0.005269 -0.606428 0.545021
C8EV_International_travel_controls_i -0.021174   0.013888 -1.524631 0.129167
C8EV_International_travel_controls_j -0.021157   0.013751 -1.538635 0.125710
                                        
log(deaths_i)                        *  
log(deaths_j)                           
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.981558     Adj. R2: 0.038294
                 Within R2: 2.777e-4

In [45]:
# Добавляю гетерогенность эффектов во времени
heterogenous_fe = feols(log(v/v19) ~ year/log(deaths_i) + year/log(deaths_j) +
year/C8EV_International_travel_controls_i + year/C8EV_International_travel_controls_j|
i + j ,
data = df)
heterogenous_fe %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v/v19)
Observations: 56,739
Fixed-effects: i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                               Estimate Std. Error   t value
year2021                                       0.078145   0.077102  1.013525
year2022                                       0.132887   0.088895  1.494875
year2020:log(deaths_i)                        -0.014540   0.006968 -2.086681
year2021:log(deaths_i)                        -0.014148   0.006784 -2.085578
year2022:log(deaths_i)                        -0.021569   0.007546 -2.858254
year2020:log(deaths_j)                        -0.010634   0.006580 -1.616124
year2021:log(deaths_j)                         0.001444   0.006831  0.211366
year2022:log(deaths_j)                         0.003974   0.007462  0.532606
year2020:C8EV_International_travel_controls_i -0.057193   0.025847 -2.212732
year2021:C8EV_International_travel_controls_i -0.026873   0.012924 -2.079314
year2022:C8EV_International

In [46]:
# С кластеризацией ошибок есть значимый эффект и наблюдается гетерогенность во времени (+ направление связи соотносится с описанным в литературе)
etable(pulled_ols, fe_no_iv, heterogenous_fe, vcov = ~i+j, signif.code=signif.code)
# А чем такая кластеризация отличается от i^j?

,,pulled_ols,fe_no_iv,heterogenous_fe
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.2953*** (0.0238),,
4,log(deaths_i),0.0062 (0.0044),-0.0156* (0.0061),
5,log(deaths_j),0.0012 (0.0037),-0.0032 (0.0053),
6,C8EV_International_travel_controls_i,-0.0445*** (0.0112),-0.0212 (0.0139),
7,C8EV_International_travel_controls_j,-0.0706*** (0.0108),-0.0212 (0.0138),
8,year2021,,,0.0781 (0.0771)
9,year2022,,,0.1329 (0.0889)


In [47]:
# При использовании НАС ошибок все эффекты становятся незначимыми
etable(pulled_ols, fe_no_iv, heterogenous_fe, vcov = vcov_NW(unit = 'pair', time = 'year'), signif.code=signif.code)

,,pulled_ols,fe_no_iv,heterogenous_fe
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.2953** (0.0131),,
4,log(deaths_i),0.0062 (0.0030),-0.0156 (0.0055),
5,log(deaths_j),0.0012 (0.0028),-0.0032 (0.0057),
6,C8EV_International_travel_controls_i,-0.0445* (0.0070),-0.0212 (0.0122),
7,C8EV_International_travel_controls_j,-0.0706* (0.0072),-0.0212 (0.0120),
8,year2021,,,0.0781 (0.0779)
9,year2022,,,0.1329 (0.0787)


In [48]:
# Всё-таки кластеризовать только по году нельзя, потому что мы в формуле указываем кусочки ошибок которые скоррелированные
    # Пробую кластеризацию ошибок по годам: с теоретической точки зрения это обосновано тем,
    # что все связанные с одной страной наблюдения связаны, значит и во всём графе наблюдения связаны.
    # При этом если нет автокорреляции (учитывая, что у меня 3 периода, это вроде ОК), то кластеризация по годам именно так и делает.
    # Но в статьях пишут, что нельзя использовать кластеризацию ошибок, если меньше чем 50 кластеров...
    # Интересно, что здесь эффект импортёра значимый, а в предыдущей спецификации эффект экспортёра...
etable(pulled_ols, fe_no_iv, heterogenous_fe, vcov = ~i+j+year, signif.code=signif.code)

,,pulled_ols,fe_no_iv,heterogenous_fe
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.2953* (0.0334),,
4,log(deaths_i),0.0062 (0.0084),-0.0156 (0.0066),
5,log(deaths_j),0.0012 (0.0036),-0.0032 (0.0036),
6,C8EV_International_travel_controls_i,-0.0445 (0.0179),-0.0212 (0.0117),
7,C8EV_International_travel_controls_j,-0.0706 (0.0288),-0.0212 (0.0139),
8,year2021,,,0.0781 (0.0622)
9,year2022,,,0.1329 (0.0834)


In [49]:
# Не понятно, насколько это осмысленно, но сделал на всякий случай
etable(pulled_ols, fe_no_iv, heterogenous_fe, vcov = ~i^year+j^year, signif.code=signif.code)

,,pulled_ols,fe_no_iv,heterogenous_fe
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.2953*** (0.0231),,
4,log(deaths_i),0.0062. (0.0037),-0.0156** (0.0049),
5,log(deaths_j),0.0012 (0.0035),-0.0032 (0.0047),
6,C8EV_International_travel_controls_i,-0.0445*** (0.0115),-0.0212. (0.0109),
7,C8EV_International_travel_controls_j,-0.0706*** (0.0113),-0.0212. (0.0113),
8,year2021,,,0.0781 (0.0624)
9,year2022,,,0.1329. (0.0701)


### Инструментальные переменные на панельных данных

In [50]:
# Pulled TSLS
# Подумать про эндогенность пропусков. Мб пропуски в инструментальных переменных можно смело заполнять?
pulled_tsls = feols(log(v/v19) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) 
 ~ 
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
+
population_density + population_density_j
  ,
 data = df
 )
pulled_tsls %>% summary(vcov = ~i+j)

NOTE: 13,906 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : H6M_Facial_Coverings_i, H6M_Facial_Coverings_j, population_density, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 56,401
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                           0.475873   0.065821  7.229792 1.4676e-11
fit_log(deaths_i)                     0.036144   0.071053  0.508685 6.1162e-01
fit_log(deaths_j)                     0.127073   0.053270  2.385467 1.8132e-02
C8EV_International_travel_controls_i -0.069961   0.022727 -3.078373 2.4189e-03
C8EV_International_travel_controls_j -0.065094   0.027706 -2.349425 1.9924e-02
                                        
(Intercept)                          ***
fit_log(deaths_i)                       
fit_log(deaths_j)                    *  
C8EV_International_travel_controls_i ** 
C8EV_Interna

In [51]:
# Только FE года (инструменты эндогенные)
fe_iv_year = feols(log(v/v19) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year|
 log(deaths_i) + log(deaths_j) ~ 
 median_age_i + median_age_j 
+
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
+
  population_density + population_density_j
 ,
 data = df
 )
fe_iv_year %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : median_age_i, median_age_j, H6M_Facial_Coverings_i, H6M_Facial_Coverings_j, population_density, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 56,401
Fixed-effects: year: 3
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
fit_log(deaths_i)                    -0.001246   0.007454 -0.167119 0.867470
fit_log(deaths_j)                    -0.006622   0.006354 -1.042260 0.298737
C8EV_International_travel_controls_i -0.000548   0.013922 -0.039367 0.968643
C8EV_International_travel_controls_j -0.027118   0.013303 -2.038542 0.043009
                                        
fit_log(deaths_i)                       
fit_log(deaths_j)                       
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.

In [52]:
# Маски эндогенные :(
feols(log(v/v19) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j|
 log(deaths_i) + log(deaths_j) ~ 
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
+
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j

 ,
 data = df
 ) %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : H6M_Facial_Coverings_i, H6M_Facial_Coverings_j, C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 56,401
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
fit_log(deaths_i)                    -0.057844   0.023691 -2.441643 0.015621
fit_log(deaths_j)                     0.009690   0.021422  0.452308 0.651610
C8EV_International_travel_controls_i -0.038695   0.018131 -2.134269 0.034221
C8EV_International_travel_controls_j -0.012589   0.013998 -0.899347 0.369711
                                        
fit_log(deaths_i)                    *  
fit_log(deaths_j)                       
C8EV_International_travel_controls_i *  
C8EV_International_travel_c

In [53]:
# Спецификация с фиксированными эффектами, куда я добавил инструменты, чтобы убрать потенциальную двустороннюю связь. 
# Добавить больше инструментов, которые меняются во времени. Мб плотность населения или медианный возраст брать не за 2019 год а в динамике, 
# чтобы использовать их как инструменты в этой спецификации.
fe_iv = feols(log(v/v19) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j|
 log(deaths_i) + log(deaths_j) ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
fe_iv %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 56,401
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
fit_log(deaths_i)                    -0.060122   0.024117 -2.492990 0.013602
fit_log(deaths_j)                     0.019263   0.021785  0.884262 0.377775
C8EV_International_travel_controls_i -0.039837   0.018595 -2.142345 0.033556
C8EV_International_travel_controls_j -0.008328   0.013863 -0.600745 0.548791
                                        
fit_log(deaths_i)                    *  
fit_log(deaths_j)                       
C8EV_International_travel_controls_i *  
C8EV_International_travel_controls_j    
---
Signif. codes:  0 '***' 0.001 

In [54]:
iv_both_endo = feols(log(v/v19) ~  1 | year+i+j|
 log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
iv_both_endo %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (IV: 13,568/10,661).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j), C8EV_International_travel_controls_i, C8EV_International_travel_controls_j
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 56,401
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                          Estimate Std. Error   t value
fit_log(deaths_i)                        -0.039481   0.028997 -1.361569
fit_log(deaths_j)                         0.023025   0.042251  0.544964
fit_C8EV_International_travel_controls_i -0.088341   0.070657 -1.250274
fit_C8EV_International_travel_controls_j -0.020243   0.080973 -0.249992
                                         Pr(>|t|) 
fit_log(deaths_i)                         0.17509 
fit_log(deaths_j)                         0.58648 
fit_C8EV_International_travel_contr

In [55]:
# Даже тест Вальда говорит, что ничего не значимо...
wald(iv_both_endo, keep = 'log|C8', vcov = ~i+j)

Wald test, H0: joint nullity of fit_log(deaths_i), fit_log(deaths_j), fit_C8EV_International_travel_controls_i and fit_C8EV_International_travel_controls_j
 stat = 1.98393, p-value = 0.093977, on 4 and 56,046 DoF, VCOV: Clustered (i & j).

In [ ]:
# Тут всё плохо в тесте Саргана
# Мы верим в синюю схему, поэтому такая спецификация бессмысленна. Утверждение в ВКР: если схема верна, то влияение отрицательное.
feols(log(v/v19) ~  1 | year|
 log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 ) %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (IV: 13,568/10,661).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j), C8EV_International_travel_controls_i, C8EV_International_travel_controls_j
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 56,401
Fixed-effects: year: 3
Standard-errors: Clustered (i & j) 
                                          Estimate Std. Error   t value
fit_log(deaths_i)                        -0.001573   0.007683 -0.204743
fit_log(deaths_j)                        -0.004590   0.007171 -0.640056
fit_C8EV_International_travel_controls_i  0.001171   0.058712  0.019943
fit_C8EV_International_travel_controls_j -0.036391   0.057637 -0.631380
                                         Pr(>|t|) 
fit_log(deaths_i)                         0.83801 
fit_log(deaths_j)                         0.52298 
fit_C8EV_International_travel_controls_i  0.98411 
fi

In [56]:
# Но если добавить температуру как инструмент, то значимость появляется...
iv_both_endo_temperature = feols(log(v/v19) ~  1 | year+i+j|
 log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
+
average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
iv_both_endo_temperature %>% summary(cluster = ~ i+j)
wald(iv_both_endo_temperature, keep = 'log|C8', vcov = ~i+j)

NOTE: 17,571 observations removed because of NA and infinite values (IV: 13,568/15,000).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j), C8EV_International_travel_controls_i, C8EV_International_travel_controls_j
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j, average_surface_temperature_i, average_surface_temperature_j
Second stage: Dep. Var.: log(v/v19)
Observations: 52,736
Fixed-effects: year: 3,  i: 166,  j: 166
Standard-errors: Clustered (i & j) 
                                          Estimate Std. Error   t value
fit_log(deaths_i)                        -0.064510   0.050350 -1.281246
fit_log(deaths_j)                         0.070782   0.035221  2.009671
fit_C8EV_International_travel_controls_i -0.049153   0.096618 -0.508742
fit_C8EV_International_travel_controls_j -0.079520   0.066498 -1.195819
                                         Pr(>|t|)    
fit_log(deaths_i)                        0.201905    
fit_log(deaths_j) 

Wald test, H0: joint nullity of fit_log(deaths_i), fit_log(deaths_j), fit_C8EV_International_travel_controls_i and fit_C8EV_International_travel_controls_j
 stat = 2.5918, p-value = 0.034688, on 4 and 52,399 DoF, VCOV: Clustered (i & j).

In [ ]:
# Как будто ограничения тоже эндогенные...
# Но в своей схеме я считаю, что они экзогенные, потому что обосновали схему.
# Если бы они были эндогенными, то наша оценка была бы смещена куда-то
iv_itc_endo = feols(log(v/v19) ~  1 + log(deaths_i) + log(deaths_j) | year+i+j|
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
iv_itc_endo %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (RHS: 6,067, IV: 9,802/10,661).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : C8EV_International_travel_controls_i, C8EV_International_travel_controls_j
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 56,401
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                          Estimate Std. Error   t value
fit_C8EV_International_travel_controls_i -0.112069   0.050736 -2.208859
fit_C8EV_International_travel_controls_j  0.012931   0.041572  0.311041
log(deaths_i)                            -0.024355   0.008355 -2.915039
log(deaths_j)                             0.000300   0.006404  0.046868
                                          Pr(>|t|)    
fit_C8EV_International_travel_controls_i 0.0284906 *  
fit_C8EV_International_travel_controls_j 0.7561421    
log(deaths_i)                            0.0040244 **

In [58]:
# Как будто ограничения тоже эндогенные...
m8 = feols(log(v/v19) ~  1 | year+i+j|
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
m8 %>% summary(cluster = ~ i+j)

NOTE: 10,661 observations removed because of NA values (IV: 9,802/10,661).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : C8EV_International_travel_controls_i, C8EV_International_travel_controls_j
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 59,646
Fixed-effects: year: 3,  i: 178,  j: 178
Standard-errors: Clustered (i & j) 
                                          Estimate Std. Error   t value
fit_C8EV_International_travel_controls_i -0.141839   0.060640 -2.339034
fit_C8EV_International_travel_controls_j  0.019681   0.049138  0.400527
                                         Pr(>|t|)    
fit_C8EV_International_travel_controls_i  0.02045 *  
fit_C8EV_International_travel_controls_j  0.68925    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.987093     Adj. R2:  0.035325
                 Within R2: -0.002872
F-test (1st stage), C8EV_International_travel_controls_i:

In [59]:
m9 = feols(log(v/v19) ~  1 | year+i+j|
 log(deaths_i) + log(deaths_j) ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
m9 %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 56,401
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                   Estimate Std. Error   t value Pr(>|t|)    
fit_log(deaths_i) -0.073065   0.029231 -2.499539 0.013362 *  
fit_log(deaths_j)  0.017674   0.024206  0.730150 0.466280    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.982531     Adj. R2:  0.035516
                 Within R2: -0.002692
F-test (1st stage), log(deaths_i): stat = 618.4    , p < 2.2e-16 , on 6 and 56,046 DoF.
F-test (1st stage), log(deaths_j): stat = 573.6    , p < 2.2e-16 , on 6 and 56,046 DoF.
                       Wu-Hausman: stat =   4.95255, p = 0.007068, on 2 and 56,046 DoF.
                           Sarg

In [ ]:
# Слово влияние это экспертное мнение на основе методов
# Температура это ложная корреляция, не надо
# Колонка 4 --- у нас нет данных, чтобы оценить эту связь. Недоидентифицированная система (всё на всё влияет)
etable(pulled_tsls, fe_iv_year, fe_iv, iv_both_endo, iv_both_endo_temperature, iv_itc_endo, vcov = ~i+j, signif.code = signif.code)

,,pulled_tsls,fe_iv_year,fe_iv,iv_both_endo,iv_both_endo_t..,iv_itc_endo
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19),log(v/v19),log(v/v19),log(v/v19)
2,,,,,,,
3,Constant,0.4759*** (0.0658),,,,,
4,log(deaths_i),0.0361 (0.0711),-0.0012 (0.0075),-0.0601* (0.0241),-0.0395 (0.0290),-0.0645 (0.0503),-0.0244** (0.0084)
5,log(deaths_j),0.1271* (0.0533),-0.0066 (0.0064),0.0193 (0.0218),0.0230 (0.0423),0.0708* (0.0352),0.0003 (0.0064)
6,C8EV_International_travel_controls_i,-0.0700** (0.0227),-0.0005 (0.0139),-0.0398* (0.0186),-0.0883 (0.0707),-0.0491 (0.0966),-0.1121* (0.0507)
7,C8EV_International_travel_controls_j,-0.0651* (0.0277),-0.0271* (0.0133),-0.0083 (0.0139),-0.0202 (0.0810),-0.0795 (0.0665),0.0129 (0.0416)
8,Fixed-Effects:,------------------,-----------------,-----------------,----------------,----------------,------------------
9,year,No,Yes,Yes,Yes,Yes,Yes


In [61]:
etable(pulled_tsls, fe_iv_year, fe_iv, iv_both_endo, iv_both_endo_temperature, iv_itc_endo, vcov = vcov_NW(unit = 'pair', time = 'year'), signif.code = signif.code)

,,pulled_tsls,fe_iv_year,fe_iv,iv_both_endo,iv_both_endo_t..,iv_itc_endo
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19),log(v/v19),log(v/v19),log(v/v19)
2,,,,,,,
3,Constant,0.4759** (0.0275),,,,,
4,log(deaths_i),0.0361 (0.0241),-0.0012 (0.0048),-0.0601 (0.0207),-0.0395 (0.0366),-0.0645 (0.0373),-0.0244. (0.0065)
5,log(deaths_j),0.1271* (0.0245),-0.0066 (0.0043),0.0193 (0.0187),0.0230 (0.0331),0.0708 (0.0278),0.0003 (0.0066)
6,C8EV_International_travel_controls_i,-0.0700* (0.0102),-0.0005 (0.0096),-0.0398 (0.0154),-0.0883 (0.0665),-0.0491 (0.0639),-0.1121 (0.0412)
7,C8EV_International_travel_controls_j,-0.0651* (0.0101),-0.0271 (0.0094),-0.0083 (0.0142),-0.0202 (0.0635),-0.0795 (0.0515),0.0129 (0.0371)
8,Fixed-Effects:,-----------------,----------------,----------------,----------------,----------------,-----------------
9,year,No,Yes,Yes,Yes,Yes,Yes


In [62]:
    # Всё ещё странно использовать кластеризацию по году... 
    # etable(m3, m4, vcov = function(x) vcovHC(x, method="arellano", type="HC1"), signif.code = signif.code) Не сработало
etable(pulled_tsls, fe_iv_year, fe_iv, iv_both_endo, iv_both_endo_temperature, iv_itc_endo, vcov = ~i+j+year, signif.code = signif.code)

,,pulled_tsls,fe_iv_year,fe_iv,iv_both_endo,iv_both_endo_t..,iv_itc_endo
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19),log(v/v19),log(v/v19),log(v/v19)
2,,,,,,,
3,Constant,0.4759 (0.1861),,,,,
4,log(deaths_i),0.0361 (0.0930),-0.0012 (0.0083),-0.0601. (0.0176),-0.0395 (0.0291),-0.0645 (0.0416),-0.0244 (0.0089)
5,log(deaths_j),0.1271 (0.0490),-0.0066 (0.0070),0.0193 (0.0195),0.0230 (0.0459),0.0708 (0.0422),0.0003 (0.0045)
6,C8EV_International_travel_controls_i,-0.0700. (0.0216),-0.0005 (0.0100),-0.0398. (0.0136),-0.0883 (0.0833),-0.0491 (0.0775),-0.1121 (0.0508)
7,C8EV_International_travel_controls_j,-0.0651 (0.0246),-0.0271 (0.0217),-0.0083 (0.0138),-0.0202 (0.1020),-0.0795 (0.1088),0.0129 (0.0396)
8,Fixed-Effects:,-----------------,----------------,-----------------,----------------,----------------,----------------
9,year,No,Yes,Yes,Yes,Yes,Yes


In [63]:
# Так вроде не очень корректно делать, если мы верим в возможную автокорреляцию
etable(pulled_tsls, fe_iv_year, fe_iv, iv_both_endo, iv_both_endo_temperature, iv_itc_endo, vcov = ~i^year+j^year, signif.code = signif.code)

,,pulled_tsls,fe_iv_year,fe_iv,iv_both_endo,iv_both_endo_t..,iv_itc_endo
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19),log(v/v19),log(v/v19),log(v/v19)
2,,,,,,,
3,Constant,0.4759*** (0.0646),,,,,
4,log(deaths_i),0.0361 (0.0558),-0.0012 (0.0052),-0.0601** (0.0204),-0.0395 (0.0255),-0.0645 (0.0399),-0.0244*** (0.0068)
5,log(deaths_j),0.1271** (0.0476),-0.0066 (0.0046),0.0193 (0.0180),0.0230 (0.0314),0.0708* (0.0281),0.0003 (0.0055)
6,C8EV_International_travel_controls_i,-0.0700** (0.0215),-0.0005 (0.0126),-0.0398** (0.0154),-0.0883 (0.0545),-0.0491 (0.0747),-0.1121** (0.0425)
7,C8EV_International_travel_controls_j,-0.0651** (0.0236),-0.0271* (0.0120),-0.0083 (0.0124),-0.0202 (0.0651),-0.0795 (0.0584),0.0129 (0.0363)
8,Fixed-Effects:,------------------,-----------------,------------------,----------------,----------------,-------------------
9,year,No,Yes,Yes,Yes,Yes,Yes


## Три пространственные регрессии (как в курсовой)

Но в такую спецификацию нельзя добавить фиксированные эффекты, поэтому почти наверное без инструментов будет смещение из-за пропущенных переменных

In [64]:
# Без инструментов.
m_2020 = feols(log(v/v19) ~  1+log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ,
 data = df %>% filter(year == '2020')
 )
m_2021 = feols(log(v/v19) ~  1+log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j,
 data = df %>% filter(year == '2021')
 )
m_2022 = feols(log(v/v19) ~  1+log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j,
 data = df %>% filter(year == '2022')
 )
summary(m_2020, vcov = 'hc1') 
summary(m_2021, vcov = 'hc1') 
summary(m_2022, vcov = 'hc1') 
etable(m_2020, m_2021, m_2022, vcov = 'hc1', signif.code = signif.code)

NOTE: 5,531 observations removed because of NA and infinite values (RHS: 5,531).

NOTE: 4,126 observations removed because of NA and infinite values (RHS: 4,126).

NOTE: 3,911 observations removed because of NA and infinite values (RHS: 3,911).



OLS estimation, Dep. Var.: log(v/v19)
Observations: 18,391
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value  Pr(>|t|)
(Intercept)                           0.025597   0.046849  0.546368 0.5848197
log(deaths_i)                         0.004091   0.003260  1.254760 0.2095820
log(deaths_j)                        -0.009842   0.003226 -3.051141 0.0022830
C8EV_International_travel_controls_i -0.016298   0.013407 -1.215616 0.2241469
C8EV_International_travel_controls_j -0.034239   0.012831 -2.668548 0.0076247
                                        
(Intercept)                             
log(deaths_i)                           
log(deaths_j)                        ** 
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.86853   Adj. R2: 8.875e-4

OLS estimation, Dep. Var.: log(v/v19)
Observations: 19,432
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                           0.121186   0.035212  3.441616 0.00057947
log(deaths_i)                         0.003676   0.004665  0.787934 0.43074466
log(deaths_j)                        -0.004846   0.004284 -1.131245 0.25796589
C8EV_International_travel_controls_i -0.002579   0.009918 -0.260020 0.79485144
C8EV_International_travel_controls_j -0.004681   0.010183 -0.459701 0.64573592
                                        
(Intercept)                          ***
log(deaths_i)                           
log(deaths_j)                           
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.0266   Adj. R2: -8.166e-5

OLS estimation, Dep. Var.: log(v/v19)
Observations: 18,916
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                           0.239995   0.019842 12.095399  < 2.2e-16
log(deaths_i)                        -0.003994   0.004750 -0.840924 0.40040089
log(deaths_j)                        -0.001500   0.004216 -0.355718 0.72205574
C8EV_International_travel_controls_i  0.021464   0.019187  1.118691 0.26328611
C8EV_International_travel_controls_j -0.063888   0.017143 -3.726715 0.00019456
                                        
(Intercept)                          ***
log(deaths_i)                           
log(deaths_j)                           
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.07671   Adj. R2: 7.389e-4

,,m_2020,m_2021,m_2022
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.0256 (0.0469),0.1212*** (0.0352),0.2400*** (0.0198)
4,log(deaths_i),0.0041 (0.0033),0.0037 (0.0047),-0.0040 (0.0047)
5,log(deaths_j),-0.0098** (0.0032),-0.0048 (0.0043),-0.0015 (0.0042)
6,C8EV_International_travel_controls_i,-0.0163 (0.0134),-0.0026 (0.0099),0.0215 (0.0192)
7,C8EV_International_travel_controls_j,-0.0342** (0.0128),-0.0047 (0.0102),-0.0639*** (0.0171)
8,____________________________________,__________________,__________________,___________________
9,S.E. type,Heteroskedas.-rob.,Heteroskedas.-rob.,Heteroskedast.-rob.


In [65]:
# С инструментами оценки коэффициентов всё равно не похожи на панельные данные (пространственные смещённые, т.к должно быть много unobservables)
# Использую ошибки hc1
# Подумать про эндогенность пропусков. Мб пропуски в инструментальных переменных можно смело заполнять?
# Почему экзогенные в панельной спецификации инструменты эндогенны в пространственной?
m_2020 = feols(log(v/v19) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) ~ 
 C1M_School_closing_i+C1M_School_closing_j+median_age_i + median_age_j + population_density_i + population_density_j,
 data = df %>% filter(year == '2020')
 )
m_2021 = feols(log(v/v19) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) ~ 
 C1M_School_closing_i+C1M_School_closing_j+ median_age_i + median_age_j + population_density_i + population_density_j,
 data = df %>% filter(year == '2021')
 )
m_2022 = feols(log(v/v19) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) ~ 
 C1M_School_closing_i+C1M_School_closing_j+ median_age_i + median_age_j + population_density_i + population_density_j,
 data = df %>% filter(year == '2022')
 )
summary(m_2020, vcov = 'hc1') 
summary(m_2021, vcov = 'hc1') 
summary(m_2022, vcov = 'hc1') 
etable(m_2020, m_2021, m_2022, vcov = 'hc1', signif.code = signif.code)
etable(m_2020, m_2021, m_2022, vcov = ~i+j, signif.code = signif.code)

NOTE: 5,531 observations removed because of NA and infinite values (RHS: 3,419, IV: 3,381/3,419).

NOTE: 4,126 observations removed because of NA and infinite values (RHS: 3,290, IV: 1,557/3,290).

NOTE: 4,249 observations removed because of NA and infinite values (RHS: 3,093, IV: 1,129/3,952).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 18,391
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value  Pr(>|t|)
(Intercept)                           0.024395   0.046827  0.520964 0.6023978
fit_log(deaths_i)                     0.007152   0.005200  1.375351 0.1690393
fit_log(deaths_j)                    -0.014607   0.004704 -3.105197 0.0019044
C8EV_International_travel_controls_i -0.015467   0.013462 -1.148868 0.2506255
C8EV_International_travel_controls_j -0.035657   0.012855 -2.773835 0.0055456
                                        
(Intercept)                             
fit_log(deaths_i)                       
fit_log(deaths_j)                    ** 
C8EV_International_travel

TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 19,432
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                           0.121400   0.035088  3.459907 0.00054152
fit_log(deaths_i)                     0.000796   0.007722  0.103102 0.91788319
fit_log(deaths_j)                    -0.002472   0.006674 -0.370455 0.71104728
C8EV_International_travel_controls_i -0.002355   0.009920 -0.237414 0.81233821
C8EV_International_travel_controls_j -0.004927   0.010189 -0.483577 0.62869129
                                        
(Intercept)                          ***
fit_log(deaths_i)                       
fit_log(deaths_j)                       
C8EV_International_

TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 18,578
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                           0.240394   0.020214 11.892533  < 2.2e-16
fit_log(deaths_i)                    -0.010058   0.005532 -1.818169 6.9054e-02
fit_log(deaths_j)                    -0.008796   0.004957 -1.774405 7.6013e-02
C8EV_International_travel_controls_i  0.014454   0.019502  0.741180 4.5859e-01
C8EV_International_travel_controls_j -0.079894   0.017737 -4.504482 6.6940e-06
                                        
(Intercept)                          ***
fit_log(deaths_i)                    .  
fit_log(deaths_j)                    .  
C8EV_International_

,,m_2020,m_2021,m_2022
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.0244 (0.0468),0.1214*** (0.0351),0.2404*** (0.0202)
4,log(deaths_i),0.0072 (0.0052),0.0008 (0.0077),-0.0101. (0.0055)
5,log(deaths_j),-0.0146** (0.0047),-0.0025 (0.0067),-0.0088. (0.0050)
6,C8EV_International_travel_controls_i,-0.0155 (0.0135),-0.0024 (0.0099),0.0144 (0.0195)
7,C8EV_International_travel_controls_j,-0.0357** (0.0128),-0.0049 (0.0102),-0.0799*** (0.0177)
8,____________________________________,__________________,__________________,___________________
9,S.E. type,Heteroskedas.-rob.,Heteroskedas.-rob.,Heteroskedast.-rob.


,,m_2020,m_2021,m_2022
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.0244 (0.0654),0.1214* (0.0492),0.2404*** (0.0446)
4,log(deaths_i),0.0072 (0.0058),0.0008 (0.0109),-0.0101 (0.0099)
5,log(deaths_j),-0.0146* (0.0061),-0.0025 (0.0093),-0.0088 (0.0084)
6,C8EV_International_travel_controls_i,-0.0155 (0.0180),-0.0024 (0.0138),0.0144 (0.0467)
7,C8EV_International_travel_controls_j,-0.0357* (0.0178),-0.0049 (0.0134),-0.0799. (0.0428)
8,____________________________________,_________________,________________,__________________
9,S.E.: Clustered,by: i & j,by: i & j,by: i & j
